# **Final Exam** 
### *Tan Yu*
***
`Here are some functions:`
####  1.  *Import and preprocess the data*

In [4]:
import csv
import pandas
import numpy as np
import operator
xtest = pandas.read_csv('bbb.test.csv', header = None)
xtrain = pandas.read_csv('bbb.train.csv',header = None)
ytest = pandas.read_csv('labels.test.csv',header = None)
ytrain = pandas.read_csv('labels.train.csv',header = None)
yytrain = ytrain.replace('nc', 1)
fytrain = yytrain.replace('c', 0)
yytest = ytest.replace('nc', 1)
fytest = yytest.replace('c', 0)

#### 2. *The Confusion Matrix Generator Function*

In [111]:
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    print('Confusion Matrix|', 'True c(0)', 'True nc(1)', 'Total')
    print('  Predict c(0)  |',  v00,'       ', v01,'       ', v00+v01)
    print('  Predict nc(1) |',  v10,'      ', v11,'       ', v10+v11)
    print('  Total         |',  v10+v00,'      ', v11+v01,'       ', v10+v11+v00+v01)

#### 3.*The prediction function*

In [9]:
def PredictFunc(x, y, beta):
    res = {}
    res['error'] = 0
    res['prey'] = []
    res['truey'] = []
    for i in range(x.shape[0]):
        temp = x.iloc[i,:]
        tempty = y.iloc[i,0]
        temppy = PredOneRow(temp, beta)
        res['prey'].append(temppy)
        res['truey'].append(tempty)
        if tempty != temppy:  
            res['error'] += 1
    res['accu'] = 1 - res['error'] / x.shape[0]
    res['mse'] = res['error'] / x.shape[0]
    return res

def PredOneRow(x, beta):
    x = np.hstack((np.ones((1)), x.T))
    pre = pi_val(np.dot(x, beta))
    if pre > 0.5:
        return 1
    return 0

#### 4.*The function to obtain $\pi_i$*

In [7]:
def pi_val(input):
    output = 1 / (1 + np.exp(-input))
    return output

***
`Here are some alternative denotations:`
***

$\beta = (\beta_0, \beta_1, ...,\beta_p)^T \\$ 
$x_i = (1, x_{i1}, ..., x_{ip})$.

***
## Problem (a) : *Log-likelihood function*
\begin{align*}
l(\beta) & = \log(L(\beta)) = \log(\prod_{i = 1}^{n} \pi_i^{y_i}(1-\pi_i)^{1 - y_i}) = \sum_{i = 1}^n y_i \log(\pi_i) + (1-y_i)\log(1-\pi_i) \\& = \sum_{i = 1}^n y_i \log(\dfrac{\pi_i}{1-\pi_i}) + \log(1-\pi_i) \\ & = \sum_{i = 1}^{n} [y_i \log(\dfrac{e^{\beta^Tx_i}}{1+e^{\beta^Tx_i}}) + (1-y_i)\log(\dfrac{1}{1+e^{\beta^Tx_i}}] \\& = \sum_{i = 1}^n y_i(\beta^Tx_i) - \log(1 + e^{\beta^Tx_i}) 
\end{align*}
***

## Problem (b) : *Logistic Regression*

### *(1) The gradient*
The parameters estimates are obtained by maximizing the above log-likelihood function. $\\$
The first-order partial derivatives of the log-likelihood for each $\beta_k, k = 0, 1, 2, ...,p$:

\begin{align*}
\dfrac{\partial l(\beta)}{\partial \beta_k} = \sum_{i=1}^n y_i x_{ik} - \pi_i x_{ik} = \sum_{i = 1}^n x_{ik}(y_i - \pi_i)
\end{align*}

### *(2) Run the model*

In [112]:
# Logistic Regression
def GradientD(x, y, stepsize, iters):
    beta = np.random.random_sample((x.shape[1],1))
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = np.dot(x.T, y - yy)
        beta = beta + stepsize*grad
    return beta

def LR(x, y, stepsize, iters):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD(x, y, stepsize, iters)
    return beta

def OutputFuncLo(beta, res):
    print('Coefficient: ', beta.flatten(), '\n')
    print('Error Num:', res['error'], '  Accuracy:', res['accu'], '\n')

betaLo = LR(xtrain, fytrain, 0.01, 500)
resLo = PredictFunc(xtest, fytest, betaLo)
OutputFuncLo(betaLo, resLo)
confusionM(resLo['truey'], resLo['prey'])

Coefficient:  [-6.95662675  5.06982462  1.74274771 -0.69012498  3.87815305  2.09302448
  0.11135059  3.04400915  1.50906571 -0.4922035  -2.72527485 -1.09253232
 -1.41684284  0.97737043  2.09496576 -0.044454    0.68156447  2.62873262
 -1.78590736 -0.90757517  0.57433208 -3.14078097  1.99495681  1.91200894
  1.69982922  0.38438736 -3.74208969 -1.19479945  1.71105622  2.82480776
 -0.05220857 -0.46149617 -0.41142094 -3.39828793] 

Error Num: 12   Accuracy: 0.8309859154929577 

Confusion Matrix| True c(0) True nc(1) Total
  Predict c(0)  | 16         1         17
  Predict nc(1) | 11        43         54
  Total         | 27        44         71


***

## Problem (c) : *Ridge Regression*
### (1) *Loss Function* 
\begin{align*}
J(\beta) = \lambda \sum_{k = 1}^p \beta_k^2  - \sum_{i = 1}^n [y_i \beta^T x_i  - \log(1 + e^{\beta^T x_i})]\end{align*}

### (2) *Explain why we use Ridge regression*

### (3) *Run the model*

 \begin{align*}\text{Gradient of Ridge Regression}:\dfrac{\partial J(\beta)}{\partial \beta_k} = 2\lambda \beta_k -\sum_{i=1}^n x_{ik}(y_i-\pi_i)\end{align*}
 

In [113]:
def GradientD3(x, y, stepsize, iters, lam3):
    beta = 0.5*np.ones((x.shape[1],1))
    #beta = np.random.random_sample((x.shape[1],1)) # 34*1
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -np.dot(x.T, y - yy)+ 2*lam3 * beta
        beta = beta - stepsize*grad
    return beta

def RR(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD3(x, y, stepsize, iters, lam3)
    return beta

def cvkfold(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = RR(xtrain_train, ytrain_train, 0.01, 100, lam3)
            restemp = PredictFunc(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse

    
def usebestlam(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] < up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = RR(xtrain, fytrain, 0.01, 100, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print('Coefficient: ', beta1.flatten(), '\n')
    print('Error Number: ', res['error'], ' Accuracy: ', res['accu'], ' Best Lambda: ', bestlam, '\n')
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.03, 0.01, 0.02, 0.04, 0.05, 0.06, 0.07]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold(data, lambdapotential, 5)
#print(lambdalist)
acc, lam = usebestlam(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest)

Coefficient:  [-3.344032    1.73033749  1.43237088  0.29798663  2.28564127  1.10387447
  0.20617683  1.8295677   1.02511576 -0.03762162 -0.93761814 -0.49178095
 -0.65697438  1.01476359  0.84972688  0.19799413  0.04467586  1.03398558
 -0.54037014 -0.3179909   0.24764916 -1.9588307   1.37560491  0.70357886
  0.77024357  0.78512616 -2.32129601 -0.59899016  0.78072296  0.9055665
  0.18767598 -0.43685696 -0.01692763 -1.72000486] 

Error Number:  12  Accuracy:  0.8309859154929577  Best Lambda:  0.03 

Confusion Matrix| True c(0) True nc(1) Total
  Predict c(0)  | 16         1         17
  Predict nc(1) | 11        43         54
  Total         | 27        44         71


***

## Problem (d) : *LASSO Regression*
### (1) *Loss Function* 
\begin{align*}
L(\beta) = \lambda \sum_{j=1}^n\left| \beta_j \right| - \sum_{i = 1}^n [y_i \beta^T x_i  - \log(1 + e^{\beta^T x_i})]  \end{align*}

### (2) *Explain why we use Lasso regression*

### (3) *Run the model*

 \begin{align*}\text{Gradient of Lasso Regression}:\dfrac{\partial L(\beta)}{\partial \beta_k} = S_{\lambda t}(\beta) -\sum_{i=1}^p x_{ik}(y_i-\pi_i)\end{align*}
 
 \begin{align*}\text{Soft-Thresholding Function}: [S_{\lambda t}(\beta)] = \left\{\begin{matrix}
\beta - t\lambda & \beta > t\lambda\\ 
0 & -t\lambda \le \beta \le t \lambda\\
\beta + t\lambda & \beta < -t\lambda  
\end{matrix}\right.\end{align*}

***

In [124]:
def soft_threshold(beta, lam):
    t = 1
    for i in range(len(beta)):
        if beta[i] > t*lam:
            beta[i] -= t*lam
        elif beta[i] < -t*lam:
            beta[i] += t*lam
        else:
            beta[i] = 0
    return beta

def GradientD4(x, y, stepsize, iters, lam3):
    beta = 0.5*np.ones((x.shape[1],1)) 
    #beta = np.random.random_sample((x.shape[1],1)) # 34*1
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -np.dot(x.T, y - yy)+ soft_threshold(beta, lam3)
        beta = beta - (stepsize*grad)
    return beta

def LA(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD4(x, y, stepsize, iters, lam3)
    return beta

def cvkfold4(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = LA(xtrain_train, ytrain_train, 0.01, 100, lam3)
            restemp = PredictFunc(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse

def usebestlam4(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] < up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = LA(xtrain, fytrain, 0.01, 300, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print('Coefficient: ', beta1.flatten(), '\n')
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam)
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.005, 0.004, 0.003, 0.0033, 0.0027]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold4(data, lambdapotential, 5)
acc, lam = usebestlam4(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest)

Coefficient:  [-2.56195010e+00  1.26246152e+00  1.11629823e+00  1.31392984e-01
  1.60138044e+00  7.16622663e-01  2.63230690e-01  1.36635774e+00
  5.71142500e-01  8.33839419e-04 -4.97083904e-01 -9.40214097e-02
 -3.28440440e-01  5.85059578e-01  3.82038926e-01  1.19003500e-01
 -2.50278938e-02  6.13503424e-01 -2.11457182e-01 -1.91839584e-03
  1.79844038e-02 -1.37618011e+00  9.35823680e-01  3.45359389e-01
  4.55240942e-01  3.60374627e-01 -1.61807132e+00 -2.97429587e-01
  4.52473156e-01  4.56870211e-01  1.20887707e-01 -1.89647706e-01
  1.65811295e-03 -1.10822119e+00] 

Accuracy:  0.8591549295774648  Best Lambda:  0.004
Confusion Matrix| True c(0) True nc(1) Total
  Predict c(0)  | 18         1         19
  Predict nc(1) | 9        43         52
  Total         | 27        44         71


## Problem (e) : *Best Model*